# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

d:\Project\face-recogtion-mlops\notebook


In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torchvision import datasets, transforms
import numpy as np
import os

d:\Project\face-recogtion-mlops\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [3]:
data_dir = './data/facenet_vn'

batch_size = 32
epochs = 1
workers = 0 if os.name == 'nt' else 8
print("workers", workers)

workers 0


#### Determine if an nvidia GPU is available

In [4]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [5]:
# help(MTCNN)

In [6]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, select_largest=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [7]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512,512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

# for i, (x, y) in enumerate(loader):
#     try:
#         mtcnn(x, save_path=y)
#         print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
#     except:
#         continue
    
# # Remove mtcnn to reduce GPU memory usage
# del mtcnn

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [8]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [9]:
from torchinfo import summary
summary(resnet)

Layer (type:depth-idx)                        Param #
InceptionResnetV1                             --
├─BasicConv2d: 1-1                            --
│    └─Conv2d: 2-1                            864
│    └─BatchNorm2d: 2-2                       64
│    └─ReLU: 2-3                              --
├─BasicConv2d: 1-2                            --
│    └─Conv2d: 2-4                            9,216
│    └─BatchNorm2d: 2-5                       64
│    └─ReLU: 2-6                              --
├─BasicConv2d: 1-3                            --
│    └─Conv2d: 2-7                            18,432
│    └─BatchNorm2d: 2-8                       128
│    └─ReLU: 2-9                              --
├─MaxPool2d: 1-4                              --
├─BasicConv2d: 1-5                            --
│    └─Conv2d: 2-10                           5,120
│    └─BatchNorm2d: 2-11                      160
│    └─ReLU: 2-12                             --
├─BasicConv2d: 1-6                            --
│ 

#### Define optimizer, scheduler, dataset, and dataloader

In [10]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [12]:
# writer = SummaryWriter()
# writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    # writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        # writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        # writer=writer
    )

# writer.close()



Initial
----------
Valid |    30/30   | loss:    7.0594 | fps:   51.0016 | acc:    0.0000   

Epoch 1/1
----------
Train |   118/118  | loss:    6.6813 | fps:   20.7464 | acc:    0.0074   
Valid |    30/30   | loss:    6.8170 | fps:   83.2964 | acc:    0.0094   


In [13]:
resnet.eval()
# Example input (dummy input to export the model)
dummy_input = torch.randn(1, 3, 224, 224)  # Example input (1 batch of 3-channel 224x224 images)

root_dir ="models/facenet_tune/"
# Export the PyTorch model to ONNX
onnx_path = root_dir+ 'resnet_model.onnx'
torch.onnx.export(resnet, dummy_input, onnx_path, verbose=True)

# Convert ONNX model to TensorFlow's .pb format using tf.compat.v1
import tensorflow as tf
import onnx

# Load ONNX model
onnx_model = onnx.load(onnx_path)

# Convert ONNX model to TensorFlow format
tf_rep = tf.compat.v1.prepare(onnx_model)

# Save TensorFlow model in .pb format
tf_pb_path = root_dir + 'model_19_6.pb'
tf_rep.export_graph(tf_pb_path)

AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'prepare'